In [60]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 500)
import plotly.graph_objects as go
import plotly.io as pio
pd.options.mode.copy_on_write = True
import math

In [61]:
bac_original = pd.read_json("drive/MyDrive/bac_2024_full_new.json")
bac_original

,nr,cod_candidat,unitate_invatamant,judet_cod,judet_nume,promotie_anterioara,forma_invatamant,specializare,limba_romana_competente,limba_romana_scris,limba_romana_contestatie,limba_romana_nota_finala,limba_materna,limba_materna_competente,limba_materna_scris,limba_materna_contestatie,limba_materna_nota_finala,limba_moderna,limba_moderna_nota,disciplina_obligatorie,disciplina_obligatorie_scris,disciplina_obligatorie_contestatie,disciplina_obligatorie_nota_finala,disciplina_aleasa,disciplina_aleasa_scris,disciplina_aleasa_contestatie,disciplina_aleasa_nota_finala,competente_digitale,media,rezultat_final
0,1,AB1471645,"COLEGIUL NATIONAL ""AVRAM IANCU"" CAMPENI",1,AB,False,Zi,Științe ale Naturii,Utilizator experimentat,10.0,NaN,10.0,None,None,None,NaN,None,Limba engleză,B2-B2-B2-B2-B2,Matematică ST-NAT,10.0,NaN,10.0,"Anatomie și fiziologie umană, genetică și ecol...",10.0,NaN,10.0,Utilizator experimentat,10.0,Reusit
1,2,B1435876,Colegiul Naţional „Grigore Moisil”,4,B,False,Zi,Matematica-Informatica,Utilizator experimentat,10.0,NaN,10.0,None,None,None,NaN,None,Limba engleză,None,Matematică MATE-INFO,10.0,NaN,10.0,Informatică MI C/C++,10.0,NaN,10.0,None,10.0,Reusit
2,3,B1438452,Colegiul Naţional „Grigore Moisil”,4,B,False,Zi,Matematica-Informatica,Utilizator experimentat,10.0,NaN,10.0,None,None,None,NaN,None,Limba engleză,None,Matematică MATE-INFO,10.0,NaN,10.0,Fizică TEO,10.0,NaN,10.0,None,10.0,Reusit
3,4,B1447238,Liceul Teoretic Internațional de Informatică B...,4,B,False,Zi,Matematica-Informatica,Utilizator experimentat,10.0,NaN,10.0,None,None,None,NaN,None,Limba engleză,None,Matematică MATE-INFO,10.0,NaN,10.0,Chimie organică TEO Nivel I/II,10.0,NaN,10.0,Utilizator experimentat,10.0,Reusit
4,5,B1484349,Colegiul Naţional „Gheorghe Lazăr”,4,B,False,Zi,Matematica-Informatica,Utilizator experimentat,10.0,NaN,10.0,None,None,None,NaN,None,Limba engleză,None,Matematică MATE-INFO,10.0,NaN,10.0,Informatică MI C/C++,10.0,NaN,10.0,Utilizator experimentat,10.0,Reusit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134415,134406,VL149043,"LICEUL PREDA BUZESCU, ORAS BERBESTI",40,VL,True,Zi,Tehnician transporturi,Utilizator avansat,4.25,NaN,4.25,None,None,None,NaN,None,Limba engleză,A2-A2--A2-A2,Matematică TEHN,Eliminat din examen,NaN,Eliminat din examen,Biologie vegetală și animală,Neprezentat,NaN,Neprezentat,Utilizator incepator,NaN,Eliminat din examen
134416,134407,VL1515294,"LICEUL TEORETIC, SAT OVESELU, COM. MĂCIUCA",40,VL,False,Zi,Științe ale Naturii,Utilizator avansat,Eliminat din examen,NaN,Eliminat din examen,None,None,None,NaN,None,Limba engleză,A2-B2-B2-B2-B2,Matematică ST-NAT,Neprezentat,NaN,Neprezentat,"Anatomie și fiziologie umană, genetică și ecol...",Neprezentat,NaN,Neprezentat,Utilizator experimentat,NaN,Eliminat din examen
134417,134408,VL1553362,"LICEUL PREDA BUZESCU, ORAS BERBESTI",40,VL,False,Zi,Tehnician mecatronist,Utilizator nivel mediu,Eliminat din examen,NaN,Eliminat din examen,None,None,None,NaN,None,Limba franceză,A2-A2-A1-A2-A2,Matematică TEHN,Neprezentat,NaN,Neprezentat,Biologie vegetală și animală,Neprezentat,NaN,Neprezentat,Utilizator incepator,NaN,Eliminat din examen
134418,134409,VL1553754,"LICEUL PREDA BUZESCU, ORAS BERBESTI",40,VL,True,Zi,Tehnician în instalații electrice,Utilizator nivel mediu,Eliminat din examen,NaN,Eliminat din examen,None,None,None,NaN,None,Limba engleză,A1-A2-A1-A1-A1,Matematică TEHN,Neprezentat,NaN,Neprezentat,Biologie vegetală și animală,Neprezentat,NaN,Neprezentat,Utilizator incepator,NaN,Eliminat din examen


In [62]:
NEGATIVE_COLOR = "red"
POSITIVE_COLOR = "#2baaff"

In [63]:
bac_for_mean = bac_original.copy()
bac_for_mean['limba_romana_nota_finala'] = pd.to_numeric(bac_for_mean['limba_romana_nota_finala'], errors='coerce')
bac_for_mean['disciplina_obligatorie_nota_finala'] = pd.to_numeric(bac_for_mean['disciplina_obligatorie_nota_finala'], errors='coerce')
bac_for_mean['disciplina_aleasa_nota_finala'] = pd.to_numeric(bac_for_mean['disciplina_aleasa_nota_finala'], errors='coerce')
bac_for_mean['limba_materna_nota_finala'] = pd.to_numeric(bac_for_mean['limba_materna_nota_finala'], errors='coerce')
bac_for_mean['media_calc'] = bac_for_mean[['limba_romana_nota_finala', 'disciplina_obligatorie_nota_finala', 'disciplina_aleasa_nota_finala', 'limba_materna_nota_finala']].mean(axis=1).apply(lambda x: x if x != x else math.trunc(x * 100) / 100)
bac_original["media_calc"] = bac_for_mean["media_calc"]

bac_for_mean_2 = bac_original.copy()
bac_for_mean_2['limba_romana_scris'] = pd.to_numeric(bac_for_mean_2['limba_romana_scris'], errors='coerce')
bac_for_mean_2['disciplina_obligatorie_scris'] = pd.to_numeric(bac_for_mean_2['disciplina_obligatorie_scris'], errors='coerce')
bac_for_mean_2['disciplina_aleasa_scris'] = pd.to_numeric(bac_for_mean_2['disciplina_aleasa_scris'], errors='coerce')
bac_for_mean_2['limba_materna_scris'] = pd.to_numeric(bac_for_mean_2['limba_materna_nota_finala'], errors='coerce')
bac_for_mean_2['media_calc_original'] = bac_for_mean_2[['limba_romana_scris', 'disciplina_obligatorie_scris', 'disciplina_aleasa_scris', 'limba_materna_scris']].mean(axis=1).apply(lambda x: x if x != x else math.trunc(x * 100) / 100)
bac_original["media_calc_original"] = bac_for_mean_2["media_calc_original"]

In [65]:
def val(bac, base_key, key_contestatie, title):
  df = pd.DataFrame()
  df["base"] = pd.to_numeric(bac[base_key], errors="coerce")
  df["contestatie"] = pd.to_numeric(bac[key_contestatie], errors="coerce")
  df.dropna(subset=["base"], inplace=True)
  df["title"] = "Disciplina " + title
  return df

def sort_by_count(x):
  return sorted(x, key=lambda a: a["contestatie"].count(), reverse=True)

VAL_ROMANA = val(bac_original, "limba_romana_scris", "limba_romana_contestatie", "Limba și literatura română")
VALS_OBLIGATORII = []
VALS_OBLIGATORII_ALL = []
for obligatorie in bac_original["disciplina_obligatorie"].unique().tolist():
  filtered = bac_original[bac_original["disciplina_obligatorie"] == obligatorie]
  data = val(filtered, "disciplina_obligatorie_scris", "disciplina_obligatorie_contestatie", obligatorie)
  if data["contestatie"].count() > 1000:
    VALS_OBLIGATORII.append(data)
  VALS_OBLIGATORII_ALL.append(data)

VALS_ALESE = []
VALS_ALESE_ALL = []
for aleasa in bac_original["disciplina_aleasa"].unique().tolist():
  filtered = bac_original[bac_original["disciplina_aleasa"] == aleasa]
  data = val(filtered, "disciplina_aleasa_scris", "disciplina_aleasa_contestatie", aleasa)
  if data["contestatie"].count() > 1000:
    VALS_ALESE.append(data)
  VALS_ALESE_ALL.append(data)

VALS_OBLIGATORII = sort_by_count(VALS_OBLIGATORII)
VALS_OBLIGATORII_ALL = sort_by_count(VALS_OBLIGATORII_ALL)

VALS_ALESE = sort_by_count(VALS_ALESE)
VALS_ALESE_ALL = sort_by_count(VALS_ALESE_ALL)

VALS_ALL = [VAL_ROMANA, *VALS_OBLIGATORII_ALL, *VALS_ALESE_ALL]
VALS_SELECTED = [VAL_ROMANA, *VALS_OBLIGATORII, *VALS_ALESE]

In [78]:
def final_results_graph(bac):
  bins = [round(x * 0.1, 1) for x in range(10, 101)]
  bac['binned'] = pd.cut(bac['media'], bins, right=False, include_lowest=True)
  bac['binned_calc'] = pd.cut(bac['media_calc'], bins, right=False, include_lowest=True)

  counts = bac['binned'].value_counts().sort_index()
  calc_counts = bac[bac['rezultat_final'] == "Respins"]['binned_calc'].value_counts().sort_index()

  bin_labels = [f"{round(interval.left, 1)}-{round(interval.right, 1)}" for interval in calc_counts.index]

  passed_line = go.Scatter(x=bin_labels,
                        y=counts.values,
                        mode="lines",
                        name="Media finală",
                        marker=dict(color=POSITIVE_COLOR)
                    )
  failed_line = go.Scatter(x=bin_labels,y=calc_counts.values, name="Media finală candidaților respinși", marker=dict(color=NEGATIVE_COLOR))

  fig = go.Figure(layout_title_text="Media finală")
  fig.add_trace(passed_line)
  fig.add_trace(failed_line)
  return fig


Adding plots for category Media finală


In [67]:
POINTS_GAINED_GRAPHS = []
for bac in VALS_SELECTED:
  bac = bac.dropna()
  bac['difference'] = bac["contestatie"] - bac["base"]

  points = go.Scatter(x=bac["base"], y=bac["difference"],
                   marker=dict(size=7, opacity=0.3, color=bac["difference"], cmin=-0.1, cmax=0.1, colorscale=[[0, NEGATIVE_COLOR], [0.5, "white"], [1, POSITIVE_COLOR]]),
                      mode="markers"
  )

  fig = go.Figure(layout_title_text=f"Punctele dobândite din contestație - {bac['title'].iloc[0]}")
  fig.update_layout(xaxis_title="Nota înainte de contestație", yaxis_title="Δ = Nota contestată - Nota inițială")
  fig.add_trace(points)
  fig.update_yaxes(ticksuffix=" puncte")
  # fig.show()
  POINTS_GAINED_GRAPHS.append(fig)
POINTS_GAINED_GRAPHS[0].show()

Adding plots for category Punctele dobândite din contestație


In [68]:
DELTA_GRAPHS = []
for bac in VALS_SELECTED:
  bac = bac.dropna()
  bac['difference'] = bac["contestatie"] - bac["base"]

  difference_positive = bac[bac["difference"] > 0]
  difference_negative = bac[bac["difference"] < 0]
  difference_same = bac[bac["difference"] == 0]

  counts_positive = difference_positive["base"].value_counts().sort_index().reset_index()
  counts_positive.columns = ["base", 'count']

  counts_negative = difference_negative["base"].value_counts().sort_index().reset_index()
  counts_negative.columns = ["base", 'count']

  counts_same = difference_same["base"].value_counts().sort_index().reset_index()
  counts_same.columns = ["base", "count"]

  positive_delta = go.Bar(
      x=counts_positive["base"],
      y=counts_positive['count'],
      name='Nr. note mai mari după contestație',
      marker=dict(color=POSITIVE_COLOR)
  )
  negative_delta = go.Bar(
      x=counts_negative["base"],
      y=counts_negative['count'],
      name='Nr. note mai mici după contestație',
      marker=dict(color=NEGATIVE_COLOR)
  )
  neutral_delta = go.Bar(
      x=counts_same["base"],
      y=counts_same['count'],
      name='Nr. note la fel după contestație',
      marker=dict(color='gray'),
  )

  fig = go.Figure(layout_title_text=f"Schimbarea notei după contestație - {bac['title'].iloc[0]}")
  fig.add_trace(positive_delta)
  fig.add_trace(negative_delta)
  fig.add_trace(neutral_delta)

  fig.update_layout(xaxis_title="Nota înainte de contestație")
  fig.update_layout(legend_title_text='Efectul contestației')
  fig.update_xaxes(nticks=10, showgrid=True)
  fig.update_yaxes(nticks=10, showgrid=True)

  fig.update_traces(width=0.04)

  fig.update_layout(xaxis=dict(range=[1,10]))
  # fig.show()
  DELTA_GRAPHS.append(fig)
DELTA_GRAPHS[0].show()

Adding plots for category Schimbarea notei după contestație


In [74]:
COUNT_GRAPHS = []
def graph_line(bac, title, percent):
  num_students = len(bac.index)

  counts = bac["base"].value_counts().sort_index().reset_index()
  counts.columns = ["base", 'count']

  fig_line = go.Scatter(x=counts["base"],
                        y=counts["count"].divide(num_students / 100 if percent else 1),
                        mode="lines",
                        name=title,
                    )

  return fig_line

for graph_input in [
    ["Limba și literatura română", [VAL_ROMANA]],
    ["Disciplina obligatorie profilului", VALS_OBLIGATORII_ALL, VALS_OBLIGATORII],
    ["Disciplina aleasă", VALS_ALESE_ALL, VALS_ALESE]
  ]:
  fig_nr_contestatii = go.Figure(layout_title_text=f"Număr de contestații versus nota inițială - {graph_input[0]}")
  fig_nr_contestatii.update_layout(xaxis_title="Nota înainte de contestație", yaxis_title="Număr de contestații")
  fig_nr_contestatii.update_layout(legend_title_text=graph_input[0])
  for bac in graph_input[1]:
    bac = bac.dropna()
    fig_nr_contestatii.add_trace(graph_line(bac, bac["title"].iloc[0], False))
  fig_nr_contestatii.show()
  COUNT_GRAPHS.append(fig_nr_contestatii)

  if len(graph_input) == 3:
    fig_nr_contestatii_percent = go.Figure(layout_title_text=f"Număr de contestații versus nota inițială - {graph_input[0]} - Procent din candidați")
    fig_nr_contestatii_percent.update_layout(xaxis_title="Nota înainte de contestație", yaxis_title="Procent din candidații disciplinei")
    fig_nr_contestatii_percent.update_layout(legend_title_text=graph_input[0])
    for bac in graph_input[2]:
      bac = bac.dropna()
      fig_nr_contestatii_percent.add_trace(graph_line(bac, bac["title"].iloc[0], True))
    fig_nr_contestatii_percent.update_yaxes(ticksuffix="%")
    fig_nr_contestatii_percent.show()
    COUNT_GRAPHS.append(fig_nr_contestatii_percent)


Adding plots for category Număr de contestații versus nota inițială


In [79]:
bac_original[bac_original["rezultat_final"] == "Respins"][bac_original["media_calc"] >= 6].sort_values(by=["media_calc"], ascending=False)

<ipython-input-79-f2ec311c0558>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,nr,cod_candidat,unitate_invatamant,judet_cod,judet_nume,promotie_anterioara,forma_invatamant,specializare,limba_romana_competente,limba_romana_scris,limba_romana_contestatie,limba_romana_nota_finala,limba_materna,limba_materna_competente,limba_materna_scris,limba_materna_contestatie,limba_materna_nota_finala,limba_moderna,limba_moderna_nota,disciplina_obligatorie,disciplina_obligatorie_scris,disciplina_obligatorie_contestatie,disciplina_obligatorie_nota_finala,disciplina_aleasa,disciplina_aleasa_scris,disciplina_aleasa_contestatie,disciplina_aleasa_nota_finala,competente_digitale,media,rezultat_final,media_calc,media_calc_original,binned,binned_calc
109953,109944,B1536797,Colegiul Naţional „Gheorghe Lazăr”,4,B,False,Zi,Științe ale Naturii,Utilizator experimentat,9.1,NaN,9.1,None,None,None,NaN,None,Limba engleză,None,Matematică ST-NAT,10.0,NaN,10.0,Fizică TEO,4.55,4.85,4.85,None,NaN,Respins,7.98,7.88,NaN,"[7.9, 8.0)"
125678,125669,SJ1455535,"LICEUL PEDAGOGIC ""GHEORGHE ȘINCAI"" ZALĂU",34,SJ,False,Zi,Filologie,Utilizator experimentat,4.9,4.75,4.75,Limba maghiară (UMAN),Utilizator experimentat,8.9,NaN,8.9,Limba engleză,B2-B2-B1-B1-B1,Istorie,8.55,NaN,8.55,Sociologie,8.05,9.35,9.35,Utilizator nivel mediu,NaN,Respins,7.88,7.60,NaN,"[7.8, 7.9)"
112674,112665,BV1548365,Colegiul Tehnic 'Mircea Cristea' Brașov,10,BV,False,Zi,Tehnician mecatronist,Utilizator avansat,9.45,NaN,9.45,None,None,None,NaN,None,Limba engleză,B2-B2-A2-B2-B2,Matematică TEHN,9.2,NaN,9.2,Biologie vegetală și animală,4.0,4.70,4.7,Utilizator nivel mediu,NaN,Respins,7.78,7.55,NaN,"[7.7, 7.8)"
119431,119422,HR1490876,"COLEGIUL TEHNIC ""BATTHYÁNY IGNÁC"" GHEORGHENI",23,HR,False,Zi,Tehnician în industria alimentară,Utilizator avansat,4.55,4.85,4.85,Limba maghiară (REAL),Utilizator experimentat,7.45,NaN,7.45,Limba engleză,A2-A2-A2-A2-B1,Matematică TEHN,9.85,NaN,9.85,"Anatomie și fiziologie umană, genetică și ecol...",8.95,NaN,8.95,Utilizator nivel mediu,NaN,Respins,7.77,7.69,NaN,"[7.7, 7.8)"
119357,119348,HR1473026,"LICEUL TEHNOLOGIC ""KÓS KÁROLY"" ODORHEIU SECUIESC",23,HR,False,Zi,Tehnician în turism,Utilizator avansat,4.7,4.60,4.6,Limba maghiară (REAL),Utilizator experimentat,8.3,NaN,8.3,Limba engleză,A2-A2-A1-A2-A2,Matematică TEHN,9.2,NaN,9.2,Geografie,8.95,NaN,8.95,Utilizator nivel mediu,NaN,Respins,7.76,7.78,NaN,"[7.7, 7.8)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116781,116772,DJ1502247,"LICEUL TEHNOLOGIC ""CONSTANTIN BRANCUSI"" CRAIOVA",18,DJ,False,Zi,Tehnician designer vestimentar,Utilizator experimentat,7.85,NaN,7.85,None,None,None,NaN,None,Limba engleză,A1-A2-A1-A2-A2,Matematică TEHN,6.95,NaN,6.95,Biologie vegetală și animală,3.2,NaN,3.2,Utilizator incepator,NaN,Respins,6.00,6.00,NaN,"[6.0, 6.1)"
120591,120582,IS1436721,"COLEGIUL AGRICOL ŞI DE INDUSTRIE ALIMENTARĂ ""V...",26,IS,False,Zi,Tehnician în agricultură ecologică,Utilizator avansat,7.25,NaN,7.25,None,None,None,NaN,None,Limba engleză,B2-B2-B2-B2-B2,Matematică TEHN,5.95,NaN,5.95,Chimie anorganică TEH Nivel I/II,4.9,4.80,4.8,Utilizator nivel mediu,NaN,Respins,6.00,6.03,NaN,"[6.0, 6.1)"
117005,116996,DJ1536468,"LICEUL ""MATEI BASARAB"" CRAIOVA",18,DJ,False,Zi,Tehnician în turism,Utilizator nivel mediu,7.8,NaN,7.8,None,None,None,NaN,None,Limba engleză,B2-B1-B2-B2-B2,Matematică TEHN,1.0,NaN,1.0,Geografie,9.2,NaN,9.2,Utilizator incepator,NaN,Respins,6.00,6.00,NaN,"[6.0, 6.1)"
117345,117336,GJ1452731,LICEUL TEHNOLOGIC BÂLTENI,19,GJ,False,Zi,Filologie,Utilizator avansat,4.7,4.55,4.55,None,None,None,NaN,None,Limba franceză,B1-B1-A1-B1-B1,Istorie,5.55,NaN,5.55,Geografie,7.9,NaN,7.9,Utilizator nivel mediu,NaN,Respins,6.00,6.05,NaN,"[6.0, 6.1)"


In [87]:
# Elevi admiși dar respinși după contestații
to_filter = bac_original[bac_original["rezultat_final"] == "Respins"][bac_original["media_calc_original"] >= 6][bac_original["media_calc"] < 6]
failed_after_contestatie = []
for index, row in to_filter.iterrows():
  if row["limba_romana_scris"] is not None and row["limba_romana_scris"] < 5:
    continue
  if row["disciplina_obligatorie_scris"] is not None and row["disciplina_obligatorie_scris"] < 5:
    continue
  if row["disciplina_aleasa_scris"] is not None and row["disciplina_aleasa_scris"] < 5:
    continue
  if row["limba_materna_scris"] is not None and row["limba_materna_scris"] < 5:
    continue
  failed_after_contestatie.append(row)


<ipython-input-87-6403e5f9c069>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-87-6403e5f9c069>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [81]:
categories_list = []
def include_plots_by_category(plots, category):
  print(f"Adding plots for category {category}")
  html = ""
  for idx, plot in enumerate(plots):
    html += pio.to_html(plot, include_plotlyjs=False, full_html=False)
  categories_list.append(dict(category=category, html=html))

include_plots_by_category([final_results_graph(bac_original)], "Media finală")
include_plots_by_category(POINTS_GAINED_GRAPHS, "Punctele dobândite din contestație")
include_plots_by_category(DELTA_GRAPHS, "Schimbarea notei după contestație")
include_plots_by_category(COUNT_GRAPHS, "Număr de contestații versus nota inițială")

Adding plots for category Media finală
Adding plots for category Punctele dobândite din contestație
Adding plots for category Schimbarea notei după contestație
Adding plots for category Număr de contestații versus nota inițială


In [99]:
from airium import Airium
a = Airium()
a('<!DOCTYPE html>')
with a.html(lang="ro"):
    with a.head():
        a.meta(charset="utf-8")
        a.title(_t="Statistici Bac 2024")
        a.meta(property="og:title", content="Statistici Bac 2024")
        a.meta(property="og:description", content="Câteva statistici despre rezultatele Bacalaureatului din 2024")
        a.script(src="https://cdn.plot.ly/plotly-2.33.0.min.js", charset="utf-8")

    with a.body():
      a.h1(_t="Statistici Bac 2024")
      with a.p():
        a("Datele au fost obținute din site-ul bacalaureat.edu.ro și pot fi descărcate aici: ")
        a.a(_t="https://github.com/tacheometry/rezultate-bac-2024", href="https://github.com/tacheometry/rezultate-bac-2024")
      with a.p():
        a("Pentru sugestii de mai multe vizualizări/etc ")
        a.a(_t="trimiteți un Issue pe GitHub", href="https://github.com/tacheometry/rezultate-bac-2024/issues/new")
        a(".")
      for category in categories_list:
          a.h2(_t=category["category"])
          a(category["html"])
      a.h2(_t="Candidați admiși inițial dar respinși după contestații")
      with a.ul():
        for x in failed_after_contestatie:
          with a.li():
            a.a(_t=x["cod_candidat"], href=f"http://bacalaureat.edu.ro/RezultateCautare.aspx?Jud={x['judet_cod']}&Cod={x['cod_candidat']}")

with open('rezultate_bac_2024_iunie_iulie.html', "wb") as f:
    f.write(bytes(a))